In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/860309
2024-09-14 11:10:03,447 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "nigeria", "city": "abuja", "street": "us-embassy", "aqicn_url": "https://api.waqi.info/feed/@13449", "latitude": 9.6, "longitude": 8.0}'

In [4]:
air_quality_fg = fs.get_feature_group(
    name = "air_quality",
    version = 1,
)

weather_fg = fs.get_feature_group(
    name = "weather",
    version = 1,
)

In [13]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df['pm25'] = aq_today_df['pm25'].astype(float)

In [14]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 180.0+ bytes


In [15]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

hourly_df

Coordinates 9.5°N 8.0°E
Elevation 737.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
date,,,,
2024-09-14 00:00:00,20.750000,0.0,2.414953,333.435028
2024-09-14 01:00:00,20.450001,0.0,2.189795,9.462248
2024-09-14 02:00:00,20.100000,0.0,2.811690,50.194473
2024-09-14 03:00:00,19.950001,0.0,3.415260,71.564964
2024-09-14 04:00:00,19.900000,0.0,3.600000,90.000000
...,...,...,...,...
2024-09-23 19:00:00,20.850000,0.8,8.280000,180.000000
2024-09-23 20:00:00,20.650000,0.8,7.280550,171.469315
2024-09-23 21:00:00,20.450001,0.8,6.830519,161.564957


In [16]:
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-09-14,24.400000,1.1,6.162207,173.290253,abuja
1,2024-09-15,27.150000,0.2,5.091168,188.130020,abuja
2,2024-09-16,28.450001,0.1,4.104631,285.255157,abuja
3,2024-09-17,27.500000,0.0,2.595997,326.309906,abuja
4,2024-09-18,26.450001,0.7,5.052841,175.914459,abuja
5,2024-09-19,27.799999,0.1,5.483356,203.198624,abuja
6,2024-09-20,28.450001,0.1,6.489992,326.309906,abuja
7,2024-09-21,28.150000,0.0,2.741678,336.801392,abuja
8,2024-09-22,25.549999,0.1,6.924738,152.102829,abuja
9,2024-09-23,23.950001,1.1,10.853866,174.289505,abuja


In [17]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


In [18]:
aq_today_df

,pm25,country,city,street,date,url
0,67.0,nigeria,abuja,us-embassy,2024-09-14,https://api.waqi.info/feed/@13449


In [19]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 180.0+ bytes


In [20]:
air_quality_fg.insert(aq_today_df)

2024-09-14 11:26:22,652 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/860309/fs/856132/fg/1181016


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/860309/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2a2928b8ef0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 613377
         }
       },
       "result": {
         "observed_value": 67.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-09-14T10:26:22.000652Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [21]:
weather_fg.insert(daily_df)

2024-09-14 11:27:02,351 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/860309/fs/856132/fg/1181017


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/860309/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2a2928bb9b0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 613378
         }
       },
       "result": {
         "observed_value": 2.595996856689453,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-09-14T10:27:02.000345Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_